In [10]:
# Description: Load tsv file

# Import libraries
import pandas as pd


# Load tsv file
News = pd.read_csv('MINDsmall_train/news.tsv', sep='\t', header=None)
News.columns = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']

#News = News.drop(16459)
#News = News.drop(2044)


User = pd.read_csv('MINDsmall_train/behaviors.tsv', sep='\t', header=None)
User.columns = ['impression_id', 'user_id', 'time', 'history', 'impressions']

User = User.dropna()

topic_size = News['category'].nunique()
subtopic_size = News['subcategory'].nunique()

print(f"Data contains {topic_size} topics and {subtopic_size} subtopics")

Data contains 17 topics and 264 subtopics


In [28]:
# Define Vocabulary for users and topics
from torchtext import vocab
from torchtext.data.utils import get_tokenizer
import torch as th
from LSTUR import GloVe

tokenizer = get_tokenizer('basic_english')

User_vocab = vocab.build_vocab_from_iterator([[id] for id in User['user_id']])
News_vocab = vocab.build_vocab_from_iterator([[id] for id in  News['news_id']])
Category_vocab = vocab.build_vocab_from_iterator([[Category] for Category in News['category']])
Subcategory_vocab = vocab.build_vocab_from_iterator([[Category] for Category in News['subcategory']])

# Define Vocabulary for title and abstract
max_title_length = max([len(tokenizer(title)) for title in News['title']])
max_history_length = max([len(history.split(" ")) for history in User['history']])
max_impressions_length = max([len(impressions.split(" ")) for impressions in User['impressions']])

# Define Datapoint to tensor
def Datapoint_to_Encodings(User):

    History = News_vocab.lookup_indices(User.history.split(" "))
    User_en = User_vocab.__getitem__(User.user_id)
    Impressions = User.impressions.split(" ")
    Impressions,Clicked = map(list, zip(*[Impression.split("-") for Impression in Impressions]))
    
    Impressions = News_vocab.lookup_indices(Impressions)
    Clicked = [int(Click) for Click in Clicked]


    History, User_en, Impressions, Clicked = map(th.tensor, [History, User_en, Impressions, Clicked])

    return History, User_en, Impressions, Clicked

# Pack Title
def pack_Title(title,max_length):

    src_len, _ = title.size()

    title_reformated = th.zeros(max_length,300)

    title_reformated[:src_len,:] = title

    return title_reformated, src_len


# Get Numeric Artikles representation
def get_Article_Encodings(Artikle):


    title = GloVe.get_vecs_by_tokens(tokenizer(Artikle['title']))
    
    #Abstract = [tokenizer(abstract) for abstract in Artikle['abstract']]
    Category = Category_vocab.__getitem__(Artikle['category'])
    Subcategory = Subcategory_vocab.__getitem__(Artikle['subcategory'])

    title, title_len = pack_Title(title,max_title_length)

    Category, Subcategory, title_len = map(th.tensor, [Category, Subcategory, title_len])

    

    return Category, Subcategory, title, title_len

# Store all News in Dictionary for faster access
News_tensors = {}

for i in range(len(News)):
    News_tensors[News_vocab.__getitem__(News['news_id'][i])] = get_Article_Encodings(News.loc[i])

# Get Numeric User representation
def Datapoint_to_tensor(User):

    History, User_en, Impressions, Clicked = Datapoint_to_Encodings(User)

    History_tensor = th.zeros(max_history_length,max_title_length,300)
    Category = th.zeros(max_history_length)
    Subcategory = th.zeros(max_history_length)
    datapoint_len = len(History)

    for idx,article in enumerate(History):
        Category[idx], Subcategory[idx], History_tensor[idx], _ = News_tensors[article.item()]

    Impressions_tensor = th.zeros(max_impressions_length,max_title_length,300)
    Category_Impressions = th.zeros(max_impressions_length)
    Subcategory_Impressions = th.zeros(max_impressions_length)
    Impressions_len = len(Impressions)

    datapoint_len, Impressions_len = map(th.tensor, [datapoint_len, Impressions_len])


    for idx,article in enumerate(Impressions):
        Category_Impressions[idx], Subcategory_Impressions[idx], Impressions_tensor[idx], _ = News_tensors[article.item()]
    
    Clicked = Clicked.argmax()+1

    return User_en, Category, Subcategory, History_tensor, datapoint_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor, Impressions_len, Clicked


In [38]:
User_en, Category, Subcategory, History_tensor, datapoint_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor, Impressions_len, Clicked = Datapoint_to_tensor(User.loc[0])


tensor(9)

In [45]:
len(User['history'][1214].split(" "))

14

In [53]:
his = [history.split(" ") for history in User['history']]

In [52]:
# Note to self. Fix Max length (Cleaning af data evt.)


max_history_length

558

In [39]:
Impressions_len

tensor(2)

In [36]:
Subcategory.shape

torch.Size([558])

In [26]:
History, User_en, Impressions, Clicked = Datapoint_to_Encodings(User.loc[0])

History_tensor = th.zeros(max_history_length,max_title_length,300)
Category = th.zeros(max_history_length)
Subcategory = th.zeros(max_history_length)
datapoint_len = len(History)

for idx,article in enumerate(History):
    Category[idx], Subcategory[idx], History_tensor[idx], _ = News_tensors[article.item()]


In [23]:
for article in History:
    print(article)

tensor(39409)
tensor(28620)
tensor(21520)
tensor(31231)
tensor(7374)
tensor(46560)
tensor(367)
tensor(8167)
tensor(19019)


In [14]:
Datapoint_to_Encodings(User.loc[0])

(tensor([39409, 28620, 21520, 31231,  7374, 46560,   367,  8167, 19019]),
 tensor(14242),
 tensor([39853, 22430]),
 tensor([1, 0]))

In [167]:
Datapoint_to_Encodings(User.loc[0])

(tensor([39409, 28620, 21520, 31231,  7374, 46560,   367,  8167, 19019]),
 tensor(14610),
 tensor([39853, 22430]),
 tensor([1, 0]))

In [154]:
News_vocab.lookup_indices(User.history[0].split(" "))

[39409, 28620, 21520, 31231, 7374, 46560, 367, 8167, 19019]

In [51]:
max_title_length

64

In [52]:

for idx,title in enumerate(News['title']):

    if len(tokenizer(title)) > 40:
        print(f"{idx}: {title}")
        print("")


1683: And the bride wore sparkles! First look at Jennifer Lawrence and Cooke Maroney as a married couple as they leave their haunted Rhode Island castle wedding venue after partying with A-list guests including Sienna Miller and Kris Jenner until 5.30am

2044: 'If I had never dialed the police, she'd still be alive': Neighbor who called a non-emergency number to ask for a 2am welfare check on a woman, 28, because her front door was open says he is horrified she was killed in her own home by a cop

3402: Billy Connolly says his Parkinson's is the 'first thing I think about when I wake up' as he reveals how he is still adapting to life with the condition but says he could make a return to live stand-up

13645: Dad-of-two, 37, drank 200 bottles of wine in two weeks at peak of booze and drugs battle that left him on the streets - but he's now turned his life around and is a Thai boxing champ

16459: The Price You Pay: The spiraling cost of college	In our new series, The Price You Pay," we 

In [61]:
News.loc[28526]['abstract']

'The five-bedroom penthouse spans three floors in one of the towers of the San Remo building, a 28-story luxury Manhattan condo built in 1930.'

In [3]:
from LSTUR import NewsEncoder, TitleEncoder, TopicEncoder, GloVe
import torch as th




topic_embedding_dim = 100
subtopic_embedding_dim = 100
word_size = 100


NewsEncoderModule = NewsEncoder(100, 100, topic_size, subtopic_size, 10000)
TitleEncoderModule = TitleEncoder(100)
TopicEncoderModule = TopicEncoder(100, 100, topic_size, subtopic_size)

topic = th.tensor([0],dtype=th.int32)
subtopic = th.tensor([0],dtype=th.int32)

title = tokenizer(News['title'][0])
W = GloVe.get_vecs_by_tokens(title)

C:\Users\Nikolaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Encode categories and subcategories:
categories = News['category'].unique()
subcategories = News['subcategory'].unique()

test = [i for i in range(len(categories))]
test2 = [i for i in range(len(subcategories))]

# Add encodings to dict
res = {}
for cat in categories:
    for value in test:
        res[cat] = value
        test.remove(value)
        break


res2 = {}
for subcat in subcategories:
    for value in test2:
        res2[subcat] = value
        test2.remove(value)
        break


# Replace column with encodings:
C = th.tensor(News['category'].replace(res))
SC = th.tensor(News['subcategory'].replace(res2))

print(C.shape)
print(SC.shape)




torch.Size([51282])
torch.Size([51282])


['the',
 'brands',
 'queen',
 'elizabeth',
 ',',
 'prince',
 'charles',
 ',',
 'and',
 'prince',
 'philip',
 'swear',
 'by']

In [99]:
# Get titles lengths
import numpy as np
from torch.nn.utils.rnn import pad_sequence

lengths = th.tensor([len(tokenizer(News['title'][i])) for i in range(News['title'].size)])


# Pad titles
maxlength = max(lengths)
padlengths = (maxlength-lengths).tolist()
padtitles = [tokenizer(News["title"][i] + " <Pad>"*padlengths[i]) for i in range(len(padlengths))]



# Embed titles
Article_embedding = th.zeros(len(padtitles),maxlength,300)
i = 0
for t in padtitles:
    Article_embedding[i] = GloVe.get_vecs_by_tokens(t)
    i+=1




In [114]:
# Dictionary of articles from their ID's
newsid = News['news_id']
article_dict = {newsid[i]:Article_embedding[i] for i in range(len(padtitles))}

print(len(article_dict))
print(article_dict["N55189"].shape)


51282
torch.Size([64, 300])


In [127]:
# User ID's of each impression log:
users = User['user_id'].unique()

test2 = [i for i in range(len(users))]


# Add encodings to dict
userid_dict = {}
for user in users:
    for value in test2:
        userid_dict[user] = value
        test2.remove(value)
        break



# Replace column with encodings:
U = th.tensor(User['user_id'].replace(userid_dict))
U.shape

KeyboardInterrupt: 

In [243]:
# For each impression log: all topics, subtopics and titles embedded and encoded in tensors
# First pad topic/subtopic sequences
N = len(User['history'])


# th.zeros()
seq_lengths = th.zeros(N)
for i in range(N):
    if isinstance(User['history'][i], str):
        seq_lengths[i] = len(tokenizer(User['history'][i]))
seq_lengths

# [item.upper() for item in test]

# seq_lengths = [(len(tokenizer(User['history'][i]))) for i in range(len(User['history']))]

# Find max sequence length
max_seq_length = max(seq_lengths)


pad_seq_lengths = ((max_seq_length-seq_lengths).to(th.int32)).tolist()

X = [item.upper() for item in User["history"][0]]
# print(tokenizer(User["history"][0]) + ["Hpad>>"]*2)



# NOT WORKING YET: NEED to encode history first
pad_hists = th.zeors(N,max_seq_length)
for i in range(N):
pad_hists = [th.tensor(tokenizer(User["history"][0]) + ["<HPad>"]*pad_seq_lengths[i]) ]

pad_hists



['n55189', 'n42782', 'n34694', 'n45794', 'n18445', 'n63302', 'n10414', 'n19347', 'n31801', 'Hpad>>', 'Hpad>>']


ValueError: too many dimensions 'str'

In [ ]:
NewsEncoderModule(topic, subtopic, W).size()

In [229]:
tokenizer(User["history"][0])

['n55189',
 'n42782',
 'n34694',
 'n45794',
 'n18445',
 'n63302',
 'n10414',
 'n19347',
 'n31801']

In [226]:
(max_seq_length-seq_lengths).to(th.int32)

tensor([549, 476, 542,  ..., 535, 500, 556], dtype=torch.int32)